import logging
logger = logging.getLogger()
assert len(logger.handlers) == 1
handler = logger.handlers[0]
handler.setLevel(logging.DEBUG)

In [3]:
import sys
sys.path.insert(0,"/home/jovyan/fairing")
print(sys.path)

['/home/jovyan/fairing', '/opt/conda/lib/python36.zip', '/opt/conda/lib/python3.6', '/opt/conda/lib/python3.6/lib-dynload', '', '/opt/conda/lib/python3.6/site-packages', '/opt/conda/lib/python3.6/site-packages/IPython/extensions', '/home/jovyan/.ipython']


In [4]:
import sys
print(sys.version)

3.6.7 | packaged by conda-forge | (default, Feb 28 2019, 09:07:38) 
[GCC 7.3.0]


In [5]:
import os
import fairing

In [6]:
# Setting up google container repositories (GCR) for storing output containers
# You can use any docker container registry istead of GCR
#GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
#DOCKER_REGISTRY = 'mtr.external.otc.telekomcloud.com/rainer_englisch'
#DOCKER_REGISTRY = 'docker.io/m1st3rb3an'
#DOCKER_REGISTRY = 'registry.hub.docker.com/m1st3rb3an'
DOCKER_REGISTRY = 'index.docker.io/m1st3rb3an'

#fairing.config.set_builder('append',base_image='tensorflow/tensorflow:latest-py3', registry=DOCKER_REGISTRY, push=True)
# Rainer: need to use latest base_image because of newer python. The older version in base image is not compatible with one in notebook in regards to serialization
#fairing.config.set_builder('append',base_image='tensorflow/tensorflow:devel-py3', registry=DOCKER_REGISTRY, push=True)
fairing.config.set_builder('append',base_image='tensorflow/tensorflow:1.13.1-py3', registry=DOCKER_REGISTRY, push=True)

fairing.config.set_deployer('job')

In [7]:
def train():
    print("hello world!")

In [8]:
"""Trains and Evaluates the MNIST network using a feed dictionary."""
import os

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist


#import fairing
#fairing.config.set_builder('docker', registry='<your-registry-name>',    base_image='tensorflow/tensorflow:1.13.1-py3')
#fairing.config.run()

INPUT_DATA_DIR = '/tmp/tensorflow/mnist/input_data/'
MAX_STEPS = 2000
BATCH_SIZE = 100
LEARNING_RATE = 0.3
HIDDEN_1 = 128
HIDDEN_2 = 32

# HACK: Ideally we would want to have a unique subpath for each instance of the job, but since we can't
# we are instead appending HOSTNAME to the logdir
LOG_DIR = os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                       'tensorflow/mnist/logs/fully_connected_feed/', os.getenv('HOSTNAME', ''))
MODEL_DIR = os.path.join(LOG_DIR, 'model.ckpt')


def train():
#    import logging
#    logger = logging.getLogger()
#    assert len(logger.handlers) == 1
#    handler = logger.handlers[0]
#    handler.setLevel(logging.DEBUG)
#    logger.warn("Start training")
    print("Print start training")

    data_sets = input_data.read_data_sets(INPUT_DATA_DIR)
    images_placeholder = tf.placeholder(
        tf.float32, shape=(BATCH_SIZE, mnist.IMAGE_PIXELS))
    labels_placeholder = tf.placeholder(tf.int32, shape=(BATCH_SIZE))

    logits = mnist.inference(images_placeholder, HIDDEN_1, HIDDEN_2)

    loss = mnist.loss(logits, labels_placeholder)
    train_op = mnist.training(loss, LEARNING_RATE)
    summary = tf.summary.merge_all()
    init = tf.global_variables_initializer()
    sess = tf.Session()
    summary_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    sess.run(init)

    data_set = data_sets.train
    for step in xrange(MAX_STEPS):
        images_feed, labels_feed = data_set.next_batch(BATCH_SIZE, False)
        feed_dict = {
            images_placeholder: images_feed,
            labels_placeholder: labels_feed,
        }

        _, loss_value = sess.run([train_op, loss],
                                 feed_dict=feed_dict)
        if step % 100 == 0:
            print("At step {}, loss = {}".format(step, loss_value))
            summary_str = sess.run(summary, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()

    logger.warn("End training")




if __name__ == '__main__':
    train()

In [10]:
# Local training
train()

Print start training
Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
At step 0, loss = 2.302619218826294


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_1' with dtype int32 and shape [100]
	 [[node Placeholder_1 (defined at <ipython-input-8-b5b0bfeb6025>:39) ]]

Caused by op 'Placeholder_1', defined at:
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/opt/conda/lib/python3.6/asyncio/base_events.py", line 427, in run_

In [ ]:
#fairing.config.run()

In [11]:
remote_train = fairing.config.fn(train)
remote_train()
#fairing.config.run()

Using preprocessor: <fairing.preprocessors.function.FunctionPreProcessor object at 0x7fe068149a58>
Using builder: <fairing.builders.append.append.AppendBuilder object at 0x7fe083b94390>
Building image using Append builder...
Creating docker context: /tmp/fairing_context_by06ecs9
/home/jovyan/fairing/fairing/__init__.py already exists in Fairing context, skipping...
Loading Docker credentials for repository 'tensorflow/tensorflow:1.13.1-py3'
Trying to load config file '/home/jovyan/.docker/config.json'
Image successfully built in 1.4147882349998326s.
Pushing image index.docker.io/m1st3rb3an/fairing-job:B0921262...
Loading Docker credentials for repository 'index.docker.io/m1st3rb3an/fairing-job:B0921262'
Trying to load config file '/home/jovyan/.docker/config.json'
Uploading index.docker.io/m1st3rb3an/fairing-job:B0921262
Layer sha256:45085432511a479b977e19d3334f4948200141d06802c222e743e3d3bafabd05 exists, skipping
Layer sha256:7e65918542624be36faca5e547a5b84c4eca21dd4e979c9309f4ad79b95